# Import necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Scan through the table and obtain the required data

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url).text
soup = BeautifulSoup(req, 'xml')
table=soup.find('table')
list_of_column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = list_of_column_names)
for tr in table.find_all('tr'):
    row_data=[]
    for td_cell in tr.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Clean the data as required

In [3]:
df=df[df['Borough']!='Not assigned']
tdf=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
tdf=tdf.reset_index(drop=False)
tdf.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
dfm = pd.merge(df, tdf, on='Postalcode')
dfm.drop(['Neighborhood'],axis=1,inplace=True)
dfm.drop_duplicates(inplace=True)
dfm.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
dfm.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
dfm.shape

(103, 3)

# P2

In [11]:
#since I was not able to download the package needed
def geocode(zipcode):
    latandlong_coords = None
    while(latandlong_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(zipdode))
        latandlong_coords = g.latandlong
    lat = latandlong_coords[0]
    long = latandlong_coords[1]
    return lat,long

In [6]:
gdf=pd.read_csv('http://cocl.us/Geospatial_data')

In [7]:
gdf.head()
gdf.rename(columns={'Postal Code':'Postalcode'},inplace=True)
gmerged = pd.merge(gdf, dfm, on='Postalcode')
gdata=gmerged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [14]:
gdata.head(100)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188


# P-3